# **Dashboard**

**Встановлення бібліотек та імпорт модулів**

In [1]:
!pip install plotly ipywidgets pandas   # встановлюємо необхідні пакети для роботи дашборду

from google.colab import output
output.enable_custom_widget_manager()   # дозволяємо використання інтерактивних віджетів у Colab

import pandas as pd      # робота з таблицями
import numpy as np       # робота з масивами, базова математика
import plotly.express as px            # швидкі інтерактивні графіки
import plotly.graph_objects as go      # більш детальні графічні об’єкти

import ipywidgets as widgets           # інтерактивні елементи (випадаючі списки тощо)
from IPython.display import display    # відображення об’єктів у ноутбуці

from google.colab import files         # інструмент для завантаження файлів у Colab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.9 MB/s eta 0:00:00


**Завантаження CSV-файлу з результатами індексу**

In [2]:
print("Завантаж, файл data_index.csv")
uploaded = files.upload()                # користувач завантажує файл вручну у Colab

df = pd.read_csv("data_index.csv")       # читаємо таблицю з інтегральними індексами
df.head()                                # перегляд перших рядків для перевірки структури


Завантаж, файл data_index.csv


Saving data_index.csv to data_index.csv


,country,iso3c,year,unemployment,trade,fdi,gov_debt,income,lending,income_num,lending_num,unemployment_z,gov_debt_z,trade_z,fdi_z,income_z,lending_z,economic_security_index,rank
0,Aruba,ABW,2005,6.0875,145.050280,-8.804936,46.73797,High income,Not classified,4,2,0.284293,0.07857,1.007981,-0.273512,1.034945,-0.048656,0.347270,32
1,Aruba,ABW,2006,6.0875,141.042627,8.920467,46.73797,High income,Not classified,4,2,0.284293,0.07857,0.935474,-0.015268,1.034945,-0.048656,0.378226,32
2,Aruba,ABW,2007,6.0875,139.972309,-17.590317,46.73797,High income,Not classified,4,2,0.284293,0.07857,0.916109,-0.401508,1.034945,-0.048656,0.310626,52
3,Aruba,ABW,2008,6.0875,139.110150,0.664176,46.73797,High income,Not classified,4,2,0.284293,0.07857,0.900511,-0.135555,1.034945,-0.048656,0.352351,46
4,Aruba,ABW,2009,6.0875,137.556576,-0.415638,46.73797,High income,Not classified,4,2,0.284293,0.07857,0.872403,-0.151287,1.034945,-0.048656,0.345045,37


**Вибір потрібного індексу та фільтрація даних**

In [3]:
# Перейменуємо індекс у коротшу назву
if "econ_security_index_step1" in df.columns:
    df["econ_index"] = df["econ_security_index_step1"]
elif "economic_security_index" in df.columns:
    df["econ_index"] = df["economic_security_index"]
else:
    raise ValueError("Не знайшов ні econ_security_index_step1, ні economic_security_index")

# приберемо рядки без індексу або ISO3 коду
df = df.dropna(subset=["econ_index", "iso3c"])

# беремо останній рядок
latest_year = int(df["year"].max())          # автоматично визначаємо найновіший доступний рік
df_latest = df[df["year"] == latest_year].copy()

print("Використовуємо рік:", latest_year)
df_latest.head()


Використовуємо рік: 2020


,country,iso3c,year,unemployment,trade,fdi,gov_debt,income,lending,income_num,lending_num,unemployment_z,gov_debt_z,trade_z,fdi_z,income_z,lending_z,economic_security_index,rank,econ_index
15,Aruba,ABW,2020,6.0875,124.137267,6.489776,46.73797,High income,Not classified,4,2,0.284293,0.078570,0.629619,-0.050681,1.034945,-0.048656,0.321348,34,0.321348
31,Afghanistan,AFG,2020,11.7100,46.709895,0.064994,46.73797,Low income,IDA,1,1,-0.729642,0.078570,-0.771212,-0.144285,-1.842381,-1.356285,-0.794206,203,-0.794206
47,Angola,AGO,2020,16.6900,65.942029,-3.848264,46.73797,Lower middle income,IBRD,2,3,-1.627712,0.078570,-0.423261,-0.201298,-0.883272,1.258973,-0.299667,148,-0.299667
63,Albania,ALB,2020,11.6900,59.520712,7.018653,83.45422,Upper middle income,IBRD,3,3,-0.726035,-2.244829,-0.539437,-0.042976,0.075836,1.258973,-0.369745,158,-0.369745
79,Andorra,AND,2020,6.0875,78.690150,9.432289,46.73797,High income,Not classified,4,2,0.284293,0.078570,-0.192619,-0.007811,1.034945,-0.048656,0.191454,53,0.191454


**Географічна мапа економічної безпеки**

In [4]:
fig_map = px.choropleth(
    df_latest,
    locations="iso3c",           # ISO3-код країни
    color="econ_index",          # значення індексу
    hover_name="country",        # ім’я країни у підказці
    hover_data={"econ_index": ":.3f", "year": True},
    color_continuous_scale="YlGnBu",   # синьо-зелена палітра
    title=f"Economic Security Index, {latest_year}"
)

fig_map.update_layout(
    coloraxis_colorbar_title="Index",
    margin=dict(l=0, r=0, t=40, b=0)
)

fig_map.show()     # показуємо інтерактивну карту


**ТОП-10 країн за економічною безпекою**

In [5]:
# Top 10
top10 = df_latest.sort_values("econ_index", ascending=False).head(10)

fig_top = px.bar(
    top10,
    x="econ_index",
    y="country",
    orientation="h",
    title="Top 10 країн за рівнем економічної безпеки",
    labels={"econ_index": "Index", "country": ""}
)
fig_top.update_layout(yaxis=dict(autorange="reversed"))  # щоби країни були зверху вниз
fig_top.show()


**10 країн з найнижчим рівнем безпеки**

In [6]:
# Bottom 10
bottom10 = df_latest.sort_values("econ_index", ascending=True).head(10)

fig_bottom = px.bar(
    bottom10,
    x="econ_index",
    y="country",
    orientation="h",
    title="Bottom 10 країн за рівнем економічної безпеки",
    labels={"econ_index": "Index", "country": ""}
)
fig_bottom.update_layout(yaxis=dict(autorange="reversed"))
fig_bottom.show()


**Підготовка індикаторів для профілю країни**

In [7]:
stim_cols = ["trade_z", "fdi_z", "income_z", "lending_z"]     # стимулятори
destim_cols = ["unemployment_z", "gov_debt_z"]                 # дестимулятори

# перевернуті дестимулятори
for col in destim_cols:
    rev_col = col + "_rev"
    if rev_col not in df.columns:
        df[rev_col] = -df[col]               # інверсія для порівняння

profile_cols = stim_cols + [c + "_rev" for c in destim_cols]


**Графік профілю індикаторів обраної країни**

In [8]:
latest_year = int(df["year"].max())
countries_latest = df[df["year"] == latest_year]["country"].sort_values().unique()

country_dropdown = widgets.Dropdown(
    options=countries_latest,
    description="Країна:",
    layout=widgets.Layout(width="50%")
)

def plot_country_profile(country):
    #  рядок для обраної країни - останній рік
    row = df[(df["country"] == country) & (df["year"] == latest_year)]
    if row.empty:
        print("Немає даних для цієї країни")
        return

    row = row.iloc[0]

    # колонки, які реально існують
    cols_existing = [c for c in profile_cols if c in df.columns]
    vals = row[cols_existing]

    df_prof = pd.DataFrame({
        "indicator": cols_existing,
        "value": vals.values
    })

    name_map = {
        "trade_z": "Trade (z)",
        "fdi_z": "FDI (z)",
        "income_z": "Income (z)",
        "lending_z": "Lending (z)",
        "unemployment_z_rev": "Unemployment (rev z)",
        "gov_debt_z_rev": "Gov debt (rev z)"
    }
    df_prof["indicator"] = df_prof["indicator"].map(
        lambda x: name_map.get(x, x)
    )

    fig = px.bar(
        df_prof,
        x="indicator",
        y="value",
        title=f"Профіль індикаторів: {country} ({latest_year})",
        labels={"indicator": "", "value": "Z-score / rev Z"},
    )
    fig.update_layout(xaxis_tickangle=-30)
    fig.show()

widgets.interact(plot_country_profile, country=country_dropdown);


interactive(children=(Dropdown(description='Країна:', layout=Layout(width='50%'), options=('Afghanistan', 'Alb…

**Таблиця країн з найвищим індексом**

In [9]:
df_latest[["country", "year", "econ_index"]].sort_values("econ_index", ascending=False).head(20)


,country,year,econ_index
2079,Malta,2020,1.222606
1327,"Hong Kong SAR, China",2020,1.079778
1871,Luxembourg,2020,1.032325
1455,Ireland,2020,0.719028
2943,Seychelles,2020,0.626077
95,United Arab Emirates,2020,0.557681
2639,Russian Federation,2020,0.553848
2367,Nauru,2020,0.550237
767,Cayman Islands,2020,0.548740
2511,Poland,2020,0.544699
